**Classificação das doenças na folhas de macieiras com fine-tunning**

In [ ]:
import pathlib
import os
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization, Flatten


In [ ]:
kernel_init = 'random_normal'

In [ ]:
dataset_folder = os.path.join("data")

In [ ]:
datasetObject = pd.read_csv(os.path.join(dataset_folder, "train.csv"))

In [ ]:
imagesDataset = pathlib.Path(os.path.join(dataset_folder, "images"))
listImages = list(imagesDataset.glob("*.*"))

In [ ]:
images_id_csv = datasetObject['image_id']
healthy_csv = datasetObject['healthy']
multiple_diseases_csv = datasetObject['multiple_diseases']
rust_csv = datasetObject['rust']
scab_csv = datasetObject['scab']

In [ ]:
images = []
labels = []
for index, values in enumerate(images_id_csv):
    img = cv2.imread(os.path.join(dataset_folder, "images", "{}.jpg".format(values)))
    img = cv2.resize(img, (100, 100))
    img = img/255
    labels.append([healthy_csv[index], multiple_diseases_csv[index], rust_csv[index],
                 scab_csv[index]])
    images.append(img)

In [ ]:
img_data_g = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=0.35,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        shear_range=0.2,
        width_shift_range=0.15,
        height_shift_range=0.15,
        rescale=None)

In [ ]:
images_after_aug = []
labels_after_aug = []
for index, image in enumerate(images):
  for i in range(9):
      img = img_data_g.flow(np.reshape(image, (1, 100, 100, 3))).next()
      images_after_aug.append(np.reshape(img, (100, 100, 3)))
      labels_after_aug.append(labels[index])
labels = np.asarray(labels_after_aug)
images = np.asarray(images_after_aug)

In [ ]:
d = tf.keras.applications.DenseNet121(include_top=False,
                   input_shape=(100,100,3),
                   pooling=None,
                   weights='imagenet')
for i in d.layers[:149]:
  i.trainable = False

for i in d.layers[149:]:
  i.trainable = True

In [ ]:
m = tf.keras.models.Sequential()
m.add(d)
m.add(tf.keras.layers.Flatten())
m.add(tf.keras.layers.BatchNormalization())
m.add(tf.keras.layers.Dense(256, activation='relu'))
m.add(tf.keras.layers.Dropout(0.7))
m.add(tf.keras.layers.BatchNormalization())
m.add(tf.keras.layers.Dense(128, activation='relu'))
m.add(tf.keras.layers.Dropout(0.5))
m.add(tf.keras.layers.BatchNormalization())
m.add(tf.keras.layers.Dense(64, activation='relu'))
m.add(tf.keras.layers.Dropout(0.3))
m.add(tf.keras.layers.Dense(4, activation='softmax'))

In [ ]:
m.compile(optimizer = tf.keras.optimizers.Adam(0.001), 
          loss = "categorical_crossentropy", 
          metrics =["accuracy", 
                     Precision(name='precision'), 
                     Recall(name='recall')])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.15, random_state = 42)

In [ ]:
history = m.fit(x_train, y_train,
                epochs=25,
                batch_size=32,
                callbacks = [
                    tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=5,
                        min_delta=0.001,
                        restore_best_weights=True)
                ])